In [1]:
import tensorflow as tf
print('Tensorflow version:', tf.__version__)
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

Tensorflow version: 2.7.0


In [2]:
# Load dataset
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

print('x_train',x_train.shape)
print('x_test',x_test.shape)

# Add channels dimension
x_train = x_train[...,tf.newaxis].astype('float32')
x_test = x_test[...,tf.newaxis].astype('float32')
print('x_train',x_train.shape)
print('x_test',x_test.shape)

x_train (60000, 28, 28)
x_test (10000, 28, 28)
x_train (60000, 28, 28, 1)
x_test (10000, 28, 28, 1)


In [3]:
# Use tf.data to batch and shuflfle the dataset
train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

2024-02-20 05:53:56.739805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-20 05:53:56.739956: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-20 05:53:56.782008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-20 05:53:56.782171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-20 05:53:56.782292: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

# Build model with Keras model subclassing API
[model subclassing API](https://www.tensorflow.org/guide/keras/making_new_layers_and_models_via_subclassing)

In [4]:
class MyModel(Model):
    def __init__(self):
        super().__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10)
    def call (self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model
model = MyModel()

In [6]:
# Choose an optimizer and loss function
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [7]:
# Select metrics to measure the loss and the accuracy of the model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

Use [tf.GradientTape](https://www.tensorflow.org/api_docs/python/tf/GradientTape) to train the model:

In [15]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)


In [18]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [19]:
EPOCHS=5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    # Train model with batch
    for images, labels in train_ds:
        train_step(images,labels)
    # Test model
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.00449703074991703, Accuracy: 99.85166931152344, Test Loss: 0.074483722448349, Test Accuracy: 98.48999786376953
Epoch 2, Loss: 0.004290905315428972, Accuracy: 99.8499984741211, Test Loss: 0.08137967437505722, Test Accuracy: 98.50999450683594
Epoch 3, Loss: 0.004437723662704229, Accuracy: 99.83666229248047, Test Loss: 0.07179128378629684, Test Accuracy: 98.55999755859375
Epoch 4, Loss: 0.003813032526522875, Accuracy: 99.89167022705078, Test Loss: 0.07635362446308136, Test Accuracy: 98.44999694824219
Epoch 5, Loss: 0.003679735818877816, Accuracy: 99.875, Test Loss: 0.07912774384021759, Test Accuracy: 98.56999969482422


# Build model with Functional API
[The Functional API](https://www.tensorflow.org/guide/keras/functional_api)